In [1]:
import ccapi

config         = ccapi.Configuration()
config.url     = "http://localhost:5000"
# config.verbose = True

### Building a Constraint-Based Model

This simple example demonstrates how to create a constraint-based model, its metabolites, reactions, genes, etc. into [Cell Collective](https://cellcollective.org).

We’ll use the `3OAS140` reaction from the `STM_1.0 model`:

```
1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] → 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]
```

Begin by importing the ccapi module into your workspace.

In [30]:
import ccapi

Now, let’s try creating a client object in order to interact with services provided by [Cell Collective](https://cellcollective.org).

In [31]:
client = ccapi.Client()

Authenticate your client using a ***password flow type authentication*** scheme.

**NOTE**: *Before you can authenticate using ccapi, you must first register an application of the appropriate type on [Cell Collective](https://cellcollective.org). If you do not require a user context, it is read only.*

In [32]:
def authenticate():
    client.auth(email = "test@cellcollective.org", password = "test")

try:
    authenticate()
except ccapi.exception.AuthenticationError:
    client.sign_up(
        email       = "test@cellcollective.org",
        password    = "test",
        first_name  = "Test",
        last_name   = "Test",
        institution = "Test"
    )
    
    authenticate()

In [33]:
user = client.me()
user

ID,1
Name,Test Test
Memory Address,0x07f153f3b4f98
First Name,Test
Last Name,Test
Email Address,test@cellcollective.org
Institution,Test


#### Creating a Base Model

Create a Base Model using ccapi and instantize it with an authenticated client.

In [35]:
model = ccapi.Model("Sample Constraint-Based Model", type_ = "metabolic", client = client)
# model.save3()
model

ID,-5319
Name,Sample Constraint-Based Model
Memory Address,0x07f153f3b4898
Number of Versions,1
Versions,[<ConstraintBasedModel -42555 version -12223 at 0x07f153f3b4320>]


A `ccapi.Model` consists of various `ccapi.ModelVersion` objects that help you build various versions to a model. By default, a `ccapi.Model` provides you a default model version of a boolean type. You can override the same by passing the `type_`.

In [36]:
# get the default model version
metabolic      = model.versions[0]
metabolic.name = "Version 1"
metabolic

ID,-42555
Version,-12223
Name,Version 1
Memory Address,0x07f153f3b4320
Number of Metabolites,0
Metabolites,
Number of Reactions,0
Reactions,


#### Adding Metabolites to a Constraint-Based Model

First, we need to create a list of metabolite objects for this model.

In [37]:
# create compartment
compartment   = ccapi.Compartment("Cytosol")

# create metabolites

malACP_c      = ccapi.Metabolite("Malonyl-acyl-carrier-protein",
                                 formula     = "C14H22N2O10PRS",
                                 compartment = compartment)
ACP_c         = ccapi.Metabolite("acyl-carrier-protein",
                                 formula     = "C11H21N2O7PRS",
                                 compartment = compartment)
h_c           = ccapi.Metabolite("H",
                                 formula     = "H",
                                 compartment = compartment)
ddcaACP_c     = ccapi.Metabolite("Dodecanoyl-ACP-n-C120ACP",
                                 formula     = "C23H43N2O8PRS",
                                 compartment = compartment)
co2_c         = ccapi.Metabolite("CO2",
                                 formula     = "CO2",
                                 compartment = compartment)
omrsACP_c     = ccapi.Metabolite("3-Oxotetradecanoyl-acyl-carrier-protein",
                                 formula     = "C25H45N2O9PRS",
                                 compartment = compartment)

Now let us add a list of components to our Constraint-Based Model.

In [38]:
metabolic.add_metabolites(malACP_c, ACP_c, h_c, ddcaACP_c, co2_c, omrsACP_c)
metabolic

ID,-42555
Version,-12223
Name,Version 1
Memory Address,0x07f153f3b4320
Number of Metabolites,6
Metabolites,"Malonyl-acyl-carrier-protein, acyl-carrier-protein, H, Dodecanoyl-ACP-n-C120ACP, CO2, 3-Oxotetradecanoyl-acyl-carrier-protein"
Number of Reactions,0
Reactions,


In [39]:
metabolic.compartments

ID,Name
-3313,Cytosol


In [40]:
metabolic.metabolites

ID,Name,Compartment,Formula,Charge
-64710,Malonyl-acyl-carrier-protein,Cytosol,C14H22N2O10PRS,
-25285,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-52385,H,Cytosol,H,
-33480,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-64851,CO2,Cytosol,CO2,
-49613,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


#### Saving a Model

Ensure you save your model in order to commit your work.

In [41]:
# model.save3()

In [42]:
metabolic.metabolites

ID,Name,Compartment,Formula,Charge
-64710,Malonyl-acyl-carrier-protein,Cytosol,C14H22N2O10PRS,
-25285,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-52385,H,Cytosol,H,
-33480,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-64851,CO2,Cytosol,CO2,
-49613,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


#### Adding Reactions to a Model

First, we create a `ccapi.Reaction` object.

In [43]:
reaction = ccapi.Reaction("3 oxoacyl acyl carrier protein synthase n C140",
                         subsystem   = "Cell Envelope Biosynthesis",
                         lower_bound = 0,
                         upper_bound = 1000)

Adding metabolites to a reaction requires using a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [44]:
reaction.add_metabolites({
    malACP_c:  -1,
    h_c:       -1,
    ddcaACP_c: -1,
    co2_c:      1,
    ACP_c:      1,
    omrsACP_c:  1
})

We will now add a reaction to the model

In [45]:
metabolic.add_reaction(reaction)

In [46]:
# model.save3()

In [47]:
metabolic.reactions

ID,Name,Number of Positive Regulators,Number of Negative Regulators,Subsystem,Lower Bound,Upper Bound
-62861,3 oxoacyl acyl carrier protein synthase n C140,0,0,Cell Envelope Biosynthesis,0,1000


#### Adding a Gene Reaction Rule to a Reaction

In order to add a Gene Reaction Rule to a Reaction, create genes as follows.

In [48]:
STM2378 = ccapi.Gene("STM2378")
STM1197 = ccapi.Gene("STM1197")

Let's consider a Gene Reaction Rule as follows

```
STM2378 or STM1197
```

You can add the same using the Regulatory Mechanism provided for Boolean Models as follows:

In [49]:
reaction.add_regulators(
    ccapi.PositiveRegulator(STM2378),
    ccapi.PositiveRegulator(STM1197)
)

In [51]:
model.save3()

ID,8
Name,Sample Constraint-Based Model
Memory Address,0x07f153f3b4898
Number of Versions,1
Versions,[<ConstraintBasedModel 8 version 9 at 0x07f153f3b4320 name='Version 1'>]


In [52]:
client.request('GET', 'api/model/8').json()

{'version': '2.7.0',
 'id': 'dcbe6cb9e50647b89bb66ae866856f59',
 'status': 'success',
 'code': 200,
 'data': {'id': 8,
  'name': 'Sample Constraint-Based Model',
  'type': 'metabolic',
  'versions': [{'id': 9,
    'metabolites': [{'compartment': 12,
      'id': 1853,
      'name': 'Malonyl-acyl-carrier-protein',
      'speciesId': 1853,
      'formula': 'C14H22N2O10PRS',
      'charge': None},
     {'compartment': 12,
      'id': 1854,
      'name': 'acyl-carrier-protein',
      'speciesId': 1854,
      'formula': 'C11H21N2O7PRS',
      'charge': None},
     {'compartment': 12,
      'id': 1855,
      'name': 'H',
      'speciesId': 1855,
      'formula': 'H',
      'charge': None},
     {'compartment': 12,
      'id': 1856,
      'name': 'Dodecanoyl-ACP-n-C120ACP',
      'speciesId': 1856,
      'formula': 'C23H43N2O8PRS',
      'charge': None},
     {'compartment': 12,
      'id': 1857,
      'name': 'CO2',
      'speciesId': 1857,
      'formula': 'CO2',
      'charge': None},
     

#### Creating a Version

Let's first create a copy of our Model Version object.

In [53]:
copy = metabolic.copy()
copy

ID,-39030
Version,9
Name,Version 1
Memory Address,0x07f153f3b81d0
Number of Metabolites,6
Metabolites,"Malonyl-acyl-carrier-protein, acyl-carrier-protein, H, Dodecanoyl-ACP-n-C120ACP, CO2, 3-Oxotetradecanoyl-acyl-carrier-protein"
Number of Reactions,1
Reactions,3 oxoacyl acyl carrier protein synthase n C140


Let's change any one of the metabolite names.

In [54]:
metabolite      = copy.metabolites[0]
metabolite.name = "foobar"

copy.metabolites

ID,Name,Compartment,Formula,Charge
-33412,foobar,Cytosol,C14H22N2O10PRS,
-1681,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-23963,H,Cytosol,H,
-38847,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-27716,CO2,Cytosol,CO2,
-31037,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


Add this new version to your model.

In [55]:
model.add_version(copy)

In [56]:
model.save3()

2020-01-29 10:53:14,337 | ERROR | Error recieved from the server: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Cannot PUT /api/model/8</pre>
</body>
</html>



HTTPError: 404 Client Error: Not Found for url: http://localhost:5000/api/model/8

#### Dynamic Linking [Experimental]

Let's assume that you'd like to reuse certain components within a model.

In [57]:
model     = ccapi.Model("Test", type_ = "metabolic", client = client)
# model.save3()

In [58]:
metabolic = model.versions[0]
metabolic

ID,-31689
Version,-30692
Name,
Memory Address,0x07f153f2e0278
Number of Metabolites,0
Metabolites,
Number of Reactions,0
Reactions,


In [59]:
compartment = ccapi.Compartment(compartment_id = 1) # Linking to an existing Compartment
metabolite  = ccapi.Metabolite("test", compartment = compartment)

metabolic.add_metabolite(metabolite)

In [60]:
model.save3()

ID,9
Name,Test
Memory Address,0x07f153f2e0240
Number of Versions,1
Versions,[<ConstraintBasedModel 9 version 10 at 0x07f153f2e0278>]
